In [ ]:
# 1. Clone the repository (BoxMOT supports StrongSORT)
!git clone https://github.com/mikel-brostrom/yolo_tracking.git
%cd yolo_tracking

# 2. Install dependencies
!pip install -v -e .  # Install the package in editable mode
!pip install ultralytics  # Ensure you have the YOLO library

fatal: destination path 'yolo_tracking' already exists and is not an empty directory.
/content/yolo_tracking
Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
Obtaining file:///content/yolo_tracking
  Running command pip subprocess to install build dependencies
  Using pip 24.1.2 from /usr/local/lib/python3.12/dist-packages/pip (python 3.12)
  Non-user install by explicit request
  Created build tracker: /tmp/pip-build-tracker-mk3542gt
  Entered build tracker: /tmp/pip-build-tracker-mk3542gt
  Created temporary directory: /tmp/pip-install-s32_uxbk
  Created temporary directory: /tmp/pip-ephem-wheel-cache-tg7pgxu8
  1 location(s) to search for versions of hatchling:
  * https://pypi.org/simple/hatchling/
  Fetching project page and analyzing links: https://pypi.org/simple/hatchling/
  Getting page https://pypi.org/simple/hatchling/
  Found index url https://pypi.org/simple/
  Looking up "https://pypi.org/simple/hatchling/" in the cache
  Request header has 

In [ ]:
import os
# Run the tracking command
# --tracking-method: strongsort
# --source: your video file
# --yolo-model: your custom .pt file
# --save: saves the video with bounding boxes drawn
# Note: We use 'boxmot track' directly, not 'python track.py'
!boxmot track \
    --tracking-method strongsort \
    --source /content/test.mp4 \
    --yolo-model /content/yolov8n.pt \
    --conf 0.25 \
    --classes 0 \
    --save \
    --save-txt

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
WARNING ⚠️ Environment does not support cv2.imshow() or PIL Image.show()

2025-12-03 21:05:04.619 | MainProcess/MainThread | INFO     | /content/yolo_tracking/boxmot/trackers/basetracker.py:56 | __init__ - BaseTracker initialization parameters:
2025-12-03 21:05:04.619 | MainProcess/MainThread | INFO     | /content/yolo_tracking/boxmot/trackers/basetracker.py:57 | __init__ - det_thresh: 0.3
2025-12-03 21:05:04.619 | MainProcess/MainThread | INFO     | /content/yolo_tracking/boxmot/trackers/basetracker.py:58 | __init__ - max_age: 30
2025-12-03 21:05:04.620 | MainProcess/MainThread | INFO     | /content/yolo_tracking/boxmot/trackers/basetracker.py:59 | __init__ - max_obs: 50
2025-12-

In [ ]:
import os
import pandas as pd
import glob

# --- CONFIGURATION ---
# Path to the folder with the million text files
# Usually: runs/track/exp.../labels/ or runs/track/exp.../tracks/
# CHECK YOUR FILE BROWSER TO BE SURE!
LABELS_FOLDER = '/content/yolo_tracking/runs/track/labels'
OUTPUT_CSV = '/content/tracking_results.csv'
# ---------------------

print(f"📂 Reading files from: {LABELS_FOLDER}")
txt_files = glob.glob(os.path.join(LABELS_FOLDER, "*.txt"))
print(f"📄 Found {len(txt_files)} text files. Parsing...")

all_data = []

for file_path in txt_files:
    # 1. Get Frame ID from filename (e.g., 'image_000135.txt' -> 135)
    filename = os.path.basename(file_path)
    try:
        # Extract number from filename
        frame_id = int(''.join(filter(str.isdigit, filename)))
    except ValueError:
        frame_id = -1 # Fallback if filename has no number

    # 2. Read the file
    with open(file_path, 'r') as f:
        lines = f.readlines()

    # 3. Parse each line
    for line in lines:
        parts = line.strip().split()
        if len(parts) >= 6:
            # YOLO Tracking Format: class x y w h id
            cls_id = int(parts[0])
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])
            track_id = int(parts[5]) # The Identity Number

            all_data.append({
                "Frame": frame_id,
                "Track_ID": track_id,
                "Class": cls_id,
                "X": x_center,
                "Y": y_center,
                "W": width,
                "H": height
            })

# 4. Save to CSV
if all_data:
    df = pd.DataFrame(all_data)
    df = df.sort_values(by=["Frame", "Track_ID"]) # Sort for easier reading
    df.to_csv(OUTPUT_CSV, index=False)
    print(f"✅ Success! Saved all tracks to: {OUTPUT_CSV}")
    print(df.head()) # Show preview
else:
    print("❌ No data found in text files.")

📂 Reading files from: /content/yolo_tracking/runs/track/labels
📄 Found 4766 text files. Parsing...
✅ Success! Saved all tracks to: /content/tracking_results.csv
      Frame  Track_ID  Class         X         Y         W         H
3501      3         1      0  0.790829  0.586523  0.054942  0.097759
1843      4         1      0  0.790858  0.586373  0.055002  0.097961
6332      5         1      0  0.790812  0.586602  0.054697  0.097369
143       6         1      0  0.790882  0.586196  0.055199  0.098275
4138      7         1      0  0.790886  0.586167  0.055225  0.098343
